# Parallelize HPO of XGBoost with Optuna and Dask

In [ ]:
# Coiled account
ACCOUNT = "dask-engineering"
# Location of feature table
FILEPATH = "s3://coiled-datasets/prefect-dask/nyc-uber-lyft/feature_table.parquet"
# Dask worker instance size
WORKER_INSTANCE_TYPE = "m6i.xlarge"
# Dict for collecting clusters for shutting down when done
CLUSTERS = {}

In [ ]:
from datetime import datetime
import threading
import joblib
from concurrent.futures import ThreadPoolExecutor

from distributed import Client, wait
import dask.dataframe as dd
import coiled

import dask
import numpy as np
import pandas as pd
import optuna
from dask_ml.metrics import mean_squared_error
import xgboost as xgb
from xgboost.dask import DaskDMatrix

import dask.dataframe as dd
import matplotlib.pyplot as plt

### Train Model

In [ ]:
# Here we subset data for cross-validation
def make_cv_splits(df, num_folds):
    frac = [1 / num_folds] * num_folds
    splits = df.random_split(frac, shuffle=True)
    for i in range(num_folds):
        train = [splits[j] for j in range(num_folds) if j != i]
        test = splits[i]
        yield train, test

In [ ]:
def train_model(trial_number, study_params, n_splits=5):
    thread_id = threading.get_ident()
    cluster_name = "xgb-nyc-taxi-gbh-" + str(thread_id)

    cluster = CLUSTERS.get(cluster_name, None)
    cluster = coiled.Cluster(
        worker_vm_types=[WORKER_INSTANCE_TYPE],
        scheduler_vm_types=["m6i.2xlarge"],
        package_sync=True,  # copy local packages,
        name=cluster_name,
        shutdown_on_close=False,  # reuse cluster across runs
        show_widget=False,
        n_workers=20,
        use_best_zone=True,
        account=ACCOUNT,
        backend_options={"region": "us-east-2", "spot": True},
        scheduler_options={"idle_timeout": "5 minutes"},
    )

    CLUSTERS[cluster_name] = cluster

    with Client(cluster) as client:
        with client.as_current():

            # Load and pre-process the DataFrame
            ddf = dd.read_parquet(FILEPATH)
            categorical_vars = ddf.select_dtypes(include="category").columns.tolist()
            ddf = ddf.categorize(columns=categorical_vars)
            float_cols = ddf.select_dtypes(include="float").columns.tolist()
            ddf[float_cols] = ddf[float_cols].astype(np.float32).persist()

            val_scores = []

            for train, test in make_cv_splits(ddf, n_splits):
                train = dd.concat(train)

                y_train = train["trip_time"].to_frame().persist()
                X_train = train.drop(columns=["trip_time"]).persist()

                # Make the test data
                y_test = test["trip_time"].to_frame().persist()
                X_test = test.drop(columns="trip_time").persist()
                dtrain = DaskDMatrix(client, X_train, y_train, enable_categorical=True)

                model = xgb.dask.train(
                    client,
                    {"tree_method": "hist", **study_params},
                    dtrain,
                    num_boost_round=4,
                    evals=[(dtrain, "train")],
                )
                predictions = xgb.dask.predict(client, model, X_test)

                score = mean_squared_error(
                    y_test.to_dask_array(lengths=True).reshape(
                        -1,
                    ),
                    predictions.to_dask_array(lengths=True),
                    squared=False,
                )
                val_scores.append(score)
            return np.mean(val_scores)

In [ ]:
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 75, 125),
        "learning_rate": trial.suggest_float("learning_rate", 0.5, 0.7),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1),
        "colsample_bynode": trial.suggest_float("colsample_bynode", 0.5, 1),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.5, 1),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 1),
        "max_depth": trial.suggest_int("max_depth", 1, 6),
        "max_leaves": trial.suggest_int("max_leaves", 0, 2),
        "max_cat_to_onehot": trial.suggest_int("max_cat_to_onehot", 1, 10),
    }
    rmse = train_model(
        trial_number=trial.number,
        study_params=params,
    )
    return rmse

In [ ]:
# Create a single study
start = datetime.now()
study = optuna.create_study(study_name="parallel-nyc-travel-time-model")

with ThreadPoolExecutor(10) as executor:
    futures = [
        executor.submit(study.optimize, objective, n_trials=5) for _ in range(10)
    ]
print(f"Total time:  {datetime.now() - start}")

In [ ]:
# Tear down running clusters

for c in CLUSTERS.values():
    c.shutdown()

In [ ]:
len(study.trials)

In [ ]:
study.best_params

In [ ]:
study.best_value

In [ ]:
# Uncomment this if you want to save the results of your study to examine later.

joblib.dump(study, "data/study_many_threads.pkl")

In [ ]:
fig = optuna.visualization.matplotlib.plot_optimization_history(study)
fig.legend(loc="upper right")
plt.savefig("data/optimization_history_study_2.png")